In [2]:
import numpy as np
import re
import pandas as pd
from pathlib import Path
import MeCab
import json
import requests


def clean_text(text):
    replaced_text = re.sub(r'[【】]', ' ', text)       # 【】の除去
    replaced_text = re.sub(r'[・_]', '', replaced_text)       # ・ の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    replaced_text = re.sub(r'\d+', '', replaced_text) # 数字の除去
    replaced_text = re.sub(r'[-/。,、.=]', ' ', replaced_text)
    return replaced_text

stop_words_ja = []
STOPWORD_FILE = Path("/home/iftekhar/amiebot/Helpbot/amie-HelpBot/amie_helpbot/assets/learning/stop_word.txt")
with open(STOPWORD_FILE, encoding='utf-8') as fr:
    stop_w = fr.read().splitlines()

newlist = ['ます',  'する', 'でき',  'です', 'ad', 'ください', 'この', 'あり',  'または', 'なり', 'その', 'ルール',]
stop_words_ja = stop_w + newlist

In [2]:
questions_word = []
questions_word_file = Path("/home/iftekhar/AI-system/JPBERT/questions_delimiter.txt")
with open(questions_word_file, encoding='utf-8') as f:
    questions_word_list = f.read().splitlines()


### Just Example how does it do the encode

In [3]:
df = pd.read_csv("~/AI-system/retrieval_Model/processed_perPage_perLine.csv")
df.head()

,Data,PageID
0,カスタマイザーからこのタイトルを設定できます。カスタマイザーからこのサブタイトルを設定できま...,0
1,Apple製品MobiControl v14 ManualApple製品をMobiContr...,1
2,Android端末MobiControl v14 ManualAndroidの設定モードA....,2
3,Windows PCMobiControl v14 ManualWindows PCの根本的...,3
4,Windows EmbeddedMobiControl v14 ManualWindows ...,4


In [4]:
# df = df.sample(frac=1).reset_index(drop=True)

In [5]:
df.head()

,Data,PageID
0,カスタマイザーからこのタイトルを設定できます。カスタマイザーからこのサブタイトルを設定できま...,0
1,Apple製品MobiControl v14 ManualApple製品をMobiContr...,1
2,Android端末MobiControl v14 ManualAndroidの設定モードA....,2
3,Windows PCMobiControl v14 ManualWindows PCの根本的...,3
4,Windows EmbeddedMobiControl v14 ManualWindows ...,4


In [6]:
#sample = df.head(20)
#sample

In [ ]:
df.iloc[[0]]
df.loc[0][0]

In [8]:
particles = ['る', '「 」', '「', '」', 'が', 'て', 'を', 'し', 'は', 
                   'か','に','の', 'へ', 'と', 'や', 'も', 'で'] 
df.Data = df.Data.apply(lambda x: clean_text(str(x)))

wakati = MeCab.Tagger("-Owakati")
df.Data = df.Data.apply(lambda x: wakati.parse(x).strip("\n"))
df.Data = df.Data.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words_ja]))
df.Data = df.Data.apply(lambda x: ' '.join(ch for ch in x.split() if ch not in particles))


In [10]:
df.head()
df.to_csv("/home/iftekhar/AI-system/retrieval_Model/processed_perPage_perLine.csv")

In [10]:
corpus_list = []
whole_corpus = []
n=100
word_limit = 10
count = 0
for index, row in df.iterrows():
    var = str(row['Data']).split('。')
    
    for elements in var:
        if len(elements) > n:
            elements = elements.split()
            # print(elements)
            # break
            chunks = [' '.join(elements[i:i+word_limit]) for i in range(0, len(elements), word_limit)]
            # print("splitted", chunks)
            # break
            for items in chunks:
                corpus_list.append([items,row['PageID']])            
        else:
            # print("not split",elements)
            corpus_list.append([elements,row['PageID']])
        # break
        
    # break    
    whole_corpus.append(corpus_list)
    corpus_list = []

In [ ]:
whole_corpus[1]

In [16]:
pages_list = []
content_list = []

count=0
for pages in whole_corpus:
    for sentences in pages:
        pages_list.append(sentences[0])
    content_list.append(pages_list)
    pages_list = []    
    

In [46]:
# This portion for splitting the corpus
split_content_corpus=[]
for pages in whole_corpus:
    for sentences in pages:
        pages_list.append([sentences[0],sentences[1]])
    split_content_corpus.append(pages_list)
    pages_list = []


len(split_content_corpus)
# for saving my split lines corpus
ids=[]
contents=[]
for i in range(len(split_content_corpus)):
    j=0
    for j in range(len(split_content_corpus[i])):
        ids.append(split_content_corpus[i][j][0])
        contents.append(split_content_corpus[i][j][1])

corpus_dataframe = pd.DataFrame(zip(contents, ids), columns=['IDs', 'Content'])
corpus_dataframe.to_csv("10words_line_corpus.csv")
corpus_dataframe.tail(100)

,IDs,Content
39089,443,端末 拾得 によって Exchange サーバ アクセス な 情報 漏洩 リスク
39090,443,無くす Exchange ActiveSync Filter インストール コンソール 該当...
39091,443,現われ た 画面 アイコ ン プルダウン 現われ Exchange アクセス 阻止
39092,443,クリック MobiControl サーバ Exchange ActiveSync Filte...
39093,443,送ら れ バー 全 OS タブ 選択 タブ サーバ タブ
...,...,...
39184,444,ページ 参照 オプション サーバ 機能 設定 端末 設定 端末 業務
39185,444,アプリ バージョンアップ VPP アカウント 設定 セキュリティ 対策 設定 分野 構成
39186,444,プロファイル 詳細 設定 端末 働きかける 端末 画面 メッセージ 表示 リモート
39187,444,ロック 紛失 モード リモート Wipe 初期 端末 働きかける 端末 状況


In [ ]:
content_list[0]

In [54]:
len(content_list[1])

219

In [55]:
    len(content_list[5])

20

In [56]:
len(content_list)

445

In [132]:
count = 0
all_question_list = []
for pages in content_list:
    question_list = []
    
    for sent in pages:
        sent = str(sent)
        
        if re.search(r'\w+', sent):
            #  print(count,"___",sent)
            item_list = []
            content = wakati.parse(sent).strip("\n").rstrip()
            item_list = content.split()
            item_list = list(set(item_list))
            #print(item_list)
            random_item_from_list = random.choice(item_list)
            item_list.remove(random_item_from_list)
            
            question = random.choice(questions_word_list)
            random.shuffle(item_list)
            item_list.append(question)
            question = ' '.join(item_list)
            question_list.append([question,count])
    count+=1
    all_question_list.append(question_list)
            
#         break   
#     break
        
        

In [133]:
len(all_question_list)

445

In [134]:
len(all_question_list[0])

127

In [ ]:
all_question_list[0]

In [140]:
labels = []
text_list = []
for i in all_question_list:
    for j in i:
        text_list.append(j[0])
        labels.append(j[1])
dataset = pd.DataFrame(zip(text_list, labels), columns=['Question','PageID'])
dataset.head()

,Question,PageID
0,タイトル 順序 MobiControl 設定 サブタイトル 関する情報を教えてください,0
1,設定 対象 順序 端末 オンプレミス 行い OS 入れられますか,0
2,MobiControl 設定 コンピュータ iOS 端末 必要 事項 macOS サーバ どうして,0
3,Domain 必須 Service 接続 サーバ 作成 Directory プロファイル A...,0
4,ADFS AD Service Federation Azure Directory 間 入...,0


In [ ]:
len(dataset)

In [204]:

def api_request(query):
    api_url = 'http://127.0.0.1:5000/api/v1.0/predict/'

    response = requests.post(api_url, 
                  data=[('output_dir', '/home/iftekhar/amiebot/Resources/876/model'),
                        ('data_dir', '/home/iftekhar/amiebot/Resources/876/data'),
                        ('agent_id', '-1'),
                        ('query', query),
                        ('max_answer', '3'),
                        ('user_nm', 'abc')
                       ]
                 )
    return response

#### Mean Reciprocal Rank is a measure to evaluate systems that return a ranked list of answers to queries. Actual_answer = [10] Predicted_answer=[20, 10, 100], Score=1/2,  if Predicted_answer=[10, 20, 100], score = 1, if Predicted_answer=[20, 100, 10], score = 1/3, and Predicted_answer=[20, 1020, 100], score=0, Then cumulative score is the total_accuracy https://en.wikipedia.org/wiki/Mean_reciprocal_rank


In [241]:
def Mean_reciprocal_rank_score(actual_answer, predicted_answers):
    pos = 0
    val = 0
    for i in predicted_answers:
        if i == actual_answer and pos==0:
            val = 1
            break
        elif i == actual_answer and pos==1:
            val = 0.5
            break
        elif i == actual_answer and pos==2:
            val = 0.33
            break
        else:
            val = 0
        pos+=1
    
    return val

In [ ]:
import time 
c=0
sum_score=0
container=[]
for index, col in dataset.iterrows():
    response = api_request(str(col['Question']))
#     time.sleep(1)
    response.encoding='utf-8'
    res = response.json()
    y = json.loads(res)
    
    page_answers = [] 
    prediction_scores = []
    
#     print("Line: ", c)
    
    for i in y['page_relevance']:
        parsed_text = ' '.join(i['keywords'])       
        page_answers.append(i['doc_id'])
        prediction_scores.append(i['score'])
    
    MRR_score = Mean_reciprocal_rank_score(col['PageID'], page_answers)
    sum_score+=MRR_score
    container.append([MRR_score, col['PageID'], page_answers, prediction_scores, parsed_text, col['Question']])
#     print(MRR_score,",", col['PageID'], ",",page_answers,",",prediction_scores,",", parsed_text, ",", col['Question'])
    
#     if c>10:
#         break
    c+=1
Total_accuracy = sum_score/c  

In [250]:
    score_list=[]
    actual_answer_list = []
    page_answers_list = []
    prediction_scores = []
    prediction_text = []
    query_list = []

    for items in container:
        score_list.append(items[0])
        actual_answer_list.append(items[1])
        page_answers_list.append(items[2])
        prediction_scores.append(items[3])
        prediction_text.append(items[4])
        query_list.append(items[5])

    d = pd.DataFrame()
    d['score'] = score_list 
    d['actual_answer'] = actual_answer_list
    d['page_answers'] = page_answers_list
    d['prediction_scores'] = prediction_scores
    d['prediction_text'] = prediction_text
    d['query_list'] = query_list


In [252]:
d.head()
    d.to_csv('performance.csv')

In [248]:
container[0]

[1,
 0,
 [0, 188, 112],
 [21.047790171749877, 14.66680545792326, 14.636287632112722],
 'タイトル 順序 mobicontrol 設定 情報',
 'タイトル 順序 MobiControl 設定 サブタイトル 関する情報を教えてください']

In [213]:
page_answers

[15, 362, 223]

### This part was recommended in the BERT service github


##### Running the server  bert-serving-start -model_dir /tmp/english_L-12_H-768_A-12/ -num_worker=4 
https://github.com/hanxiao/bert-as-service